<a href="https://colab.research.google.com/github/josepmatias/ONE/blob/main/capitaliza%C3%A7%C3%A3o%20one%20mar%2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [1]:
pip install pyxlsb

In [7]:
capit="unit_03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas_VS3.xlsb"
folha="listagem upfront"
folha_pagar="Valor a pagar"

In [4]:
#carregar ficheiro dos alarmes

from google.colab import files
uploaded = files.upload()

Saving unit_03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas_VS3.xlsb to unit_03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas_VS3.xlsb


In [5]:
#ler folha auto

folha_capit=pd.read_excel(capit, sheet_name=folha)

In [8]:
#ler folha valor a pagar

folha_pagar=pd.read_excel(capit, sheet_name=folha_pagar)

In [61]:
#definir lista com o nome das colunas

colunas_pagar=["username", "chefe de equipa", "função", "parceiro", "Activações", "Correção autos anteriores", "Correções autos anteriores_histórico",\
"Correções autos anteriores a aplicar", "Total ativações", "Penalidades N", "Penalidades_histórico", "Penalidade total a aplicar", \
"Charge back N", "Charge back_histórico", "Charge back total a aplicar", "Ativação Negativa", "Ativação Negativa Histórico", "Ativação Negativa total a aplicar", \
"Melhoria Contínua", "Valor", "VALOR COM FATOR", "Correções a transitar", "Penalidades a transitar", "Charge back a transitar", "Ativações negativas a transitar", \
"Status CAV N-1", "Status CAV N", "Imputar ao CC do OBV_correções", "Imputar ao CC do OBV_penalidades", "Imputar ao CC do OBV_charge back", \
"Imputar ao CC do OBV_ativ neg", "Correções a transitar para o mês seguinte", "Penalidade a transitar para o mês seguinte", \
"Charge back a transitar para o mês seguinte", "Ativações negativas a transitar para o mês seguinte"]
print(len(colunas_pagar))

35


In [62]:
#definir uma nova bd
folhas_pagar2=folha_pagar.iloc[:, :len(colunas_pagar)].dropna().iloc[1:, :]
folhas_pagar2.columns=colunas_pagar
print(folhas_pagar2.shape)
print(folhas_pagar2.columns)
folhas_pagar2.head()


(203, 35)
Index(['username', 'chefe de equipa', 'função', 'parceiro', 'Activações',
       'Correção autos anteriores', 'Correções autos anteriores_histórico',
       'Correções autos anteriores a aplicar', 'Total ativações',
       'Penalidades N', 'Penalidades_histórico', 'Penalidade total a aplicar',
       'Charge back N', 'Charge back_histórico', 'Charge back total a aplicar',
       'Ativação Negativa', 'Ativação Negativa Histórico',
       'Ativação Negativa total a aplicar', 'Melhoria Contínua', 'Valor',
       'VALOR COM FATOR', 'Correções a transitar', 'Penalidades a transitar',
       'Charge back a transitar', 'Ativações negativas a transitar',
       'Status CAV N-1', 'Status CAV N', 'Imputar ao CC do OBV_correções',
       'Imputar ao CC do OBV_penalidades', 'Imputar ao CC do OBV_charge back',
       'Imputar ao CC do OBV_ativ neg',
       'Correções a transitar para o mês seguinte',
       'Penalidade a transitar para o mês seguinte',
       'Charge back a transitar para

,username,chefe de equipa,função,parceiro,Activações,Correção autos anteriores,Correções autos anteriores_histórico,Correções autos anteriores a aplicar,Total ativações,Penalidades N,...,Status CAV N-1,Status CAV N,Imputar ao CC do OBV_correções,Imputar ao CC do OBV_penalidades,Imputar ao CC do OBV_charge back,Imputar ao CC do OBV_ativ neg,Correções a transitar para o mês seguinte,Penalidade a transitar para o mês seguinte,Charge back a transitar para o mês seguinte,Ativações negativas a transitar para o mês seguinte
6,U80086295,U80037039,Operador,Heading,990.6,4.852174,0,4.852174,995.452174,0,...,ACTIVO,ACTIVO,0,0,0,0,0,0,0,0
7,U80047659,U80037039,Operador,Heading,774.6,0,0,0,774.6,-32,...,ACTIVO,ACTIVO,0,0,0,0,0,0,0,0
8,amargrodrigues,U80037039,Operador,Randstad,582.6,0,0,0,582.6,-32,...,ACTIVO,ACTIVO,0,0,0,0,0,0,0,0
9,jfcsilva,U80037039,Operador,Heading,111.75,0,0,0,111.75,-6,...,ACTIVO,ACTIVO,0,0,0,0,0,0,0,0
10,PJBARROSO,TLCOIMBRA,Operador,Heading,939.4,-5.12,0,-5.12,934.28,-32,...,ACTIVO,ACTIVO,0,0,0,0,0,0,0,0


In [73]:
#def nao_acerto(df, col1, col2="username"):
  #print("estes são os cavs onde teremos que distribuir valor pelas linhas")
  #return df.loc[df[col1]>0, col2].tolist()

def nao_acerto(df, col1, col3, col2="username"):
  correcao_total=df.loc[(df[col1]!=0) & (df[col3]==0), col2].tolist()
  correcao_ratear=df.loc[(df[col1]!=0) & (df[col3]!=0), col2].tolist()
  return correcao_total, correcao_ratear


#correcções
nao_acerto(folhas_pagar2, "Correções autos anteriores a aplicar", "Correções a transitar para o mês seguinte")
#activações negativas
nao_acerto(folhas_pagar2, "Ativação Negativa total a aplicar", "Ativações negativas a transitar para o mês seguinte")


nao_acerto(folhas_pagar2, "Charge back total a aplicar", "Charge back a transitar para o mês seguinte")

(['U80086295',
  'U80047659',
  'amargrodrigues',
  'PJBARROSO',
  'CMMACIEIRA',
  'jcamartins',
  'mncosta',
  'anmerocha',
  'U80048294',
  'asmartins',
  'ccmeirinhos',
  'U80083852',
  'fantobarbosa',
  'CPSARDON',
  'raazevedo',
  'jpconceicao',
  'acristinasantos',
  'imsampaio',
  'nacasousa',
  'drodsantos',
  'ramadeira',
  'isproenca',
  'tdleite',
  'anasoferreira',
  'LSROCHA',
  'mmmorais',
  'CRREIS',
  'dalexteixeira',
  'carobarros',
  'omgoncalves',
  'scoliveira',
  'stsousa',
  'caneribeiro',
  'aisferreira',
  'dfribeiro',
  'papesousa',
  'ifpinho',
  'lscanhota',
  'dhhenchel',
  'hemaneves',
  'tmreis',
  'crmendes',
  'gaferreira',
  'MJPEREIRA',
  'U80073949',
  'lrmaduro',
  'TLCOIMBRA',
  'SSMACHADO',
  'U80077616',
  'U80037519',
  'U80047459'],
 ['sscardoso'])